In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
from functools import reduce
import random

### UD

In [3]:
# import kaggle
# kaggle.api.competitions_list_cli()
# !kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge
# kaggle.api.competition_download_file('allen-institute-for-ai/CORD-19-research-challenge',file_name='COVID19.zip')
# kaggle.api.competition_download_file('COVID-19 Open Research Dataset Challenge (CORD-19)',file_name='COVID19.zip')

In [4]:
import numpy as np
import re
import tensorflow as tf
import string
import collections
from collections import Counter
import matplotlib.pyplot as plt

In [5]:
import json

In [6]:
import os

In [7]:
os.getcwd()

'/home/x0104908/Practice/NLP/COVID19'

In [8]:
txt = ''

In [9]:
jsons = [i for i in os.listdir(path='biorxiv_medrxiv/biorxiv_medrxiv/') if '.json' in i]

In [10]:

for d in jsons:
    one = json.load(open('biorxiv_medrxiv/biorxiv_medrxiv/' + d))
    for i in one['abstract']:
    #     print(i['text'])
        txt += i['text']

    for i in one['body_text']:
    #     print(i['text'])
        txt += i['text']

    for i,j in one['ref_entries'].items():
        if 'text' in one['ref_entries'][i].keys():
    #         print(i)
            txt += one['ref_entries'][i]['text']

In [11]:
len(txt)

28003061

In [8]:
jsons1 = [i for i in os.listdir(path='custom_license/custom_license/') if '.json' in i]

In [9]:
for d in jsons1:
    one = json.load(open('custom_license/custom_license/' + d))
    for i in one['abstract']:
    #     print(i['text'])
        txt += i['text']

    for i in one['body_text']:
    #     print(i['text'])
        txt += i['text']

    for i,j in one['ref_entries'].items():
        if 'text' in one['ref_entries'][i].keys():
    #         print(i)
            txt += one['ref_entries'][i]['text']

In [10]:
len(txt)

639179720

In [9]:
jsons2 = [i for i in os.listdir(path='comm_use_subset/comm_use_subset/') if '.json' in i]

In [10]:
for d in jsons2:
    one = json.load(open('comm_use_subset/comm_use_subset/' + d))
    for i in one['abstract']:
    #     print(i['text'])
        txt += i['text']

    for i in one['body_text']:
    #     print(i['text'])
        txt += i['text']

    for i,j in one['ref_entries'].items():
        if 'text' in one['ref_entries'][i].keys():
    #         print(i)
            txt += one['ref_entries'][i]['text']

In [11]:
len(txt)

315366190

In [12]:
jsons3 = [i for i in os.listdir(path='noncomm_use_subset/noncomm_use_subset/') if '.json' in i]

In [13]:
for d in jsons3:
    one = json.load(open('noncomm_use_subset/noncomm_use_subset/' + d))
    for i in one['abstract']:
    #     print(i['text'])
        txt += i['text']

    for i in one['body_text']:
    #     print(i['text'])
        txt += i['text']

    for i,j in one['ref_entries'].items():
        if 'text' in one['ref_entries'][i].keys():
    #         print(i)
            txt += one['ref_entries'][i]['text']

In [2]:
len(txt)

NameError: name 'txt' is not defined

In [12]:
words = txt.split()

In [13]:
len(words)

48325116

In [14]:
len(set(words))

1453674

In [15]:
unique_words = list(set(words))

In [16]:
len(unique_words)

1453674

### Subsampling

In [17]:
word2int = {}
int2word = {}
vocab_size = len(unique_words) # gives the total number of unique words
for i,word in enumerate(unique_words):
    word2int[word] = i
    int2word[i] = word

In [18]:
all_words_freq = dict(Counter(words))
all_words_fraction = {i:j/len(words) for i,j in all_words_freq.items()}
all_words_fraction['heart']


threshold = 1e-4
p_drop = {i:(1 - np.sqrt(threshold / j)) for i,j in all_words_fraction.items()}
min(p_drop.values()),max(p_drop.values())



def drop_or_not(word):
    base = random.random()
    if p_drop[word] > base:
        return True
    else:
        return False

In [19]:
vocabulary_size = len(set(words))

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

sub_samp_words = [i for i in words if drop_or_not(i) == False]
vocabulary_size = len(set(sub_samp_words))
data, count, dictionary, reverse_dictionary = build_dataset(sub_samp_words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
# del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 1], ('the', 103142), ('of', 92969), ('and', 84927), ('in', 71987)]
Sample data [157, 524, 246, 900, 21409, 575, 2429, 1752, 34, 726]


In [20]:
vocabulary_size

1453674

In [21]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['there', 'limited', 'information', 'diversity', 'co-circulating', 'humans', 'animals.', 'Here,']

with num_skips = 2 and skip_window = 1:
    batch: ['limited', 'limited', 'information', 'information', 'diversity', 'diversity', 'co-circulating', 'co-circulating']
    labels: ['information', 'there', 'diversity', 'limited', 'information', 'co-circulating', 'diversity', 'humans']

with num_skips = 4 and skip_window = 2:
    batch: ['information', 'information', 'information', 'information', 'diversity', 'diversity', 'diversity', 'diversity']
    labels: ['co-circulating', 'limited', 'diversity', 'there', 'information', 'co-circulating', 'humans', 'limited']


### Negative sampling

In [22]:
all_words_freq = dict(count)

In [23]:
numerator = np.array([all_words_freq[i]**0.75 for i in all_words_freq.keys()])
denominator = sum([all_words_freq[i]**0.75 for i in all_words_freq.keys()])

neg_samp = {dictionary[i]:(j**0.75)/denominator for i,j in all_words_freq.items()}
prob_sum = sum(neg_samp.values())
neg_samp = {i:j/prob_sum for i,j in neg_samp.items()}

for i in range(10):
    print(np.random.choice(a=list(neg_samp.keys()),size=10,replace=False,p=list(neg_samp.values())))

[  9392   7256     63   1381 464300  19215 395878    615  47804  19726]
[  21268   84186 1231132    1030   11846 1047729   48178   36349   13372
   98175]
[   9203  493296 1229106    4074     320  569942   29031   12034   34163
  306638]
[  43613 1256828     586    7789    3334     558   73975 1292921  583167
   31519]
[525218   3861 126219   4097 326503   4511   1556 680408   2665    453]
[   1227  117764  854024  657472  377999    1004      88   65482  171251
 1160289]
[   4206   14837  110388 1283392     983      38    1739   12413 1174180
     255]
[ 805697 1030432   14164    8867     391   16983     408  360019   26078
     920]
[  70469     889    6627     696   44608 1214810  531724    2382    7856
   41220]
[    254    3132   46392   62511  187348    1191  750463  121054   11714
 1039910]


In [24]:
batch_size = 128
embedding_size = 300 # Dimension of the embedding vector.
skip_window = 2 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.
unigrams = list(list(zip(*sorted(neg_samp.items(),key=lambda x: x[0])))[1])
graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  sampler = tf.nn.fixed_unigram_candidate_sampler(true_classes=tf.cast(train_labels,dtype=tf.int64),num_true=1,num_sampled=num_sampled,unique=True,
                                         range_max=vocabulary_size,distortion=0.75,num_reserved_ids=0,unigrams=unigrams,name='unigram_sampler')
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                               labels=train_labels, num_sampled=num_sampled,sampled_values= sampler,num_classes=vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [25]:
num_steps = 100000
# num_steps = 30000
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    if (data_index % len(data) == 0) & (data_index != 0):
        print("$$$$$$$$$$$$$$$$$$$$$   Resampling all words   $$$$$$$$$$$$$$$$$$$$$$$$")
        sub_samp_words = [i for i in words if drop_or_not(i) == False]
        vocabulary_size = len(set(sub_samp_words))
        data, count, dictionary, reverse_dictionary = build_dataset(sub_samp_words)
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 7.602454
Nearest to control: 9-O-and, Pandupuspitasari,, org/10.1007/s00705-016-3010-9, preempted, pentosaceus, 0.01.PK-15, WPV3-each, eps,,
Nearest to used: 9105-9117., GAG-3′), fullest,, below.>, composition.Below, cholesterol-conjugated, (technical), 갖고,
Nearest to of: trans-14g, Ruaha, 96.3, Non-amphipatic, non-Salmonella, Reinfection, 5746-TW98, 1.74×10,
Nearest to be: 6.4.22, Garcinia,, 2007b;, TZDs, (46/86)., Békés, insulinspecific, Irf8,
Nearest to was: SR-BI), Qubit5, Trx-like, intima,, Lebarbenchon, Fremonk,, nonmaleficence.Beneficence, (AGCTGCAGGC,
Nearest to this: CD11c-cells, mir-132, sequence.Acute, CC-3052-treated, 1102,, -GCTCACTTG, FeatureName, 양측검정,
Nearest to gene: -diastolic, 60S,, FCV-pos./neg., 10.1038/srep34589Common, variationsThe, April,, (Marko, 0.997355),
Nearest to at: Clofazimine,, -http://bmjopen.bmj.com/site/about/guidelines.xhtmlPolicy"., Q05320.Similarities, serially-diluted,, C394Y, qRT-PCR.Signal, fPL),, Garcelon,,


In [ ]:
# num_points = 800

# tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
# two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

# ax,fig = plt.subplots(figsize=(200,200))
# h = plt.scatter(two_d_embeddings[:,0], two_d_embeddings[:,1])
# for i in range(vocabulary_size):
#     try:
#         plt.annotate(s=reverse_dictionary[i].encode("utf8").decode("utf8"), xy=(Z[i,0], Z[i,1]))
#     except:
#         print("bad string:", reverse_dictionary[i])
# plt.savefig('COVID19_words_map.pdf')

In [27]:
num_points = 800
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [28]:
def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  pylab.figure(figsize=(200,200))  # in inches
  for i, label in enumerate(labels):
    x, y = embeddings[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
#   pylab.show()
  pylab.savefig('comm_use_subset_1_800words.pdf')

words = [reverse_dictionary[i] for i in range(1, num_points+1)]
plot(two_d_embeddings, words)

In [182]:
4251-3667

584

In [26]:
final_embeddings.shape

(532167, 300)

In [27]:
wrds = list(reverse_dictionary.values())

In [28]:
wrds.pop(0)

'immunoprivileged'

In [29]:
import pandas as pd

pd.DataFrame(final_embeddings[1:]).to_csv('noncomm_use_subset_weights.tsv',sep='\t',index=False,header=False)
pd.DataFrame(wrds).to_csv('noncomm_use_subset_words.tsv',sep='\t',index=False,header=False)

In [26]:
def analogy(pos1, neg1, pos2, neg2, word2idx, idx2word, W):
  V, D = W.shape

  # don't actually use pos2 in calculation, just print what's expected
  print("testing: %s - %s = %s - %s" % (pos1, neg1, pos2, neg2))
  for w in (pos1, neg1, pos2, neg2):
    if w not in word2idx:
      print("Sorry, %s not in word2idx" % w)
      return

  p1 = W[word2idx[pos1]]
  n1 = W[word2idx[neg1]]
  p2 = W[word2idx[pos2]]
  n2 = W[word2idx[neg2]]

  vec = p1 - n1 + n2

  distances = pairwise_distances(vec.reshape(1, D), W, metric='cosine').reshape(V)
  idx = distances.argsort()[:10]

  # pick one that's not p1, n1, or n2
  best_idx = -1
  keep_out = [word2idx[w] for w in (pos1, neg1, neg2)]
  # print("keep_out:", keep_out)
  for i in idx:
    if i not in keep_out:
      best_idx = i
      break
  # print("best_idx:", best_idx)

  print("got: %s - %s = %s - %s" % (pos1, neg1, idx2word[best_idx], neg2))
  print("closest 10:")
  for i in idx:
    print(idx2word[i], distances[i])

  print("dist to %s:" % pos2, cos_dist(p2, vec))

In [27]:
word2idx = {j:i for i,j in reverse_dictionary.items()}

In [65]:
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import cosine as cos_dist

In [66]:
def test_model(word2idx, W):
    # there are multiple ways to get the "final" word embedding
    # We = (W + V.T) / 2
    We = W
    idx2word = {i:w for w, i in word2idx.items()}
    analogy('king', 'man', 'queen', 'woman', word2idx, idx2word, We)
    analogy('king', 'prince', 'queen', 'princess', word2idx, idx2word, We)
    analogy('miami', 'florida', 'dallas', 'texas', word2idx, idx2word, We)
    analogy('einstein', 'scientist', 'picasso', 'painter', word2idx, idx2word, We)
    analogy('japan', 'sushi', 'germany', 'bratwurst', word2idx, idx2word, We)
    analogy('man', 'woman', 'he', 'she', word2idx, idx2word, We)
    analogy('man', 'woman', 'uncle', 'aunt', word2idx, idx2word, We)
    analogy('man', 'woman', 'brother', 'sister', word2idx, idx2word, We)
    analogy('man', 'woman', 'husband', 'wife', word2idx, idx2word, We)
    analogy('man', 'woman', 'actor', 'actress', word2idx, idx2word, We)
    analogy('man', 'woman', 'father', 'mother', word2idx, idx2word, We)
    analogy('heir', 'heiress', 'prince', 'princess', word2idx, idx2word, We)
    analogy('nephew', 'niece', 'uncle', 'aunt', word2idx, idx2word, We)
    analogy('france', 'paris', 'japan', 'tokyo', word2idx, idx2word, We)
    analogy('france', 'paris', 'china', 'beijing', word2idx, idx2word, We)
    analogy('february', 'january', 'december', 'november', word2idx, idx2word, We)
    analogy('france', 'paris', 'germany', 'berlin', word2idx, idx2word, We)
    analogy('week', 'day', 'year', 'month', word2idx, idx2word, We)
    analogy('week', 'day', 'hour', 'minute', word2idx, idx2word, We)
    analogy('france', 'paris', 'italy', 'rome', word2idx, idx2word, We)
    analogy('paris', 'france', 'rome', 'italy', word2idx, idx2word, We)
    analogy('france', 'french', 'england', 'english', word2idx, idx2word, We)
    analogy('japan', 'japanese', 'china', 'chinese', word2idx, idx2word, We)
    analogy('china', 'chinese', 'america', 'american', word2idx, idx2word, We)
    analogy('japan', 'japanese', 'italy', 'italian', word2idx, idx2word, We)
    analogy('japan', 'japanese', 'australia', 'australian', word2idx, idx2word, We)
    analogy('walk', 'walking', 'swim', 'swimming', word2idx, idx2word, We)

In [67]:
test_model(word2idx,final_embeddings)

testing: king - man = queen - woman
got: king - man = Showbiz - woman
closest 10:
king 0.23405427
Showbiz 0.62121505
woman 0.6335519
pegs 0.63540876
aground 0.63605785
Harper's 0.6418189
Eyeballing 0.6436876
heathens 0.6573821
shredder 0.67840385
appraising 0.6803443
dist to queen: 0.9383346773684025
testing: king - prince = queen - princess
got: king - prince = Baird's - princess
closest 10:
king 0.39990473
princess 0.4307624
Baird's 0.6278224
Department's 0.63611126
uncovers 0.64125144
FLYBY 0.6596608
fragment 0.67292535
shish 0.6731931
visionary 0.67434776
that 0.6771451
dist to queen: 0.9191022738814354
testing: miami - florida = dallas - texas
Sorry, miami not in word2idx
testing: einstein - scientist = picasso - painter
Sorry, einstein not in word2idx
testing: japan - sushi = germany - bratwurst
Sorry, japan not in word2idx
testing: man - woman = he - she
got: man - woman = he - she
closest 10:
she 0.26556325
he 0.31341237
He 0.569121
She 0.60651994
Conklin 0.63967407
man 0.64395

### Questions

#### What is known about transmission, incubation, and environmental stability?

In [29]:
valid_word = 'incubation'

In [30]:
# valid_examples = np.array(random.sample(range(valid_window), valid_size))
valid_examples = np.array([dictionary[valid_word]])

In [31]:
valid_examples

array([731])

In [32]:
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

In [33]:
valid_embeddings = tf.nn.embedding_lookup(final_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, tf.transpose(final_embeddings))

In [ ]:
with tf.Session() as session:
    sim = similarity.eval()

In [ ]:
sim.shape

In [1]:
top_k = 20 # number of nearest neighbors
nearest = (-sim[0, :]).argsort()[1:top_k+1]
log = 'Nearest to %s:' % valid_word
for k in range(top_k):
  close_word = reverse_dictionary[nearest[k]]
  log = '%s %s,' % (log, close_word)
print(log)

NameError: name 'sim' is not defined

In [ ]:
def get_surrounding_txt(word,txt):
    match = re.compile(r'{}'.format(word))
    match.search(word)


In [ ]:
def get_surrounding_txt(word,txt):
    match = re.compile(r'{}'.format(word))

In [247]:
word = 'transmission'

In [248]:
[(m.start(0), m.end(0)) for m in re.finditer(r'{}'.format(word), txt)]

[(31578, 31590),
 (33187, 33199),
 (37732, 37744),
 (37828, 37840),
 (37873, 37885),
 (153500, 153512),
 (174852, 174864),
 (180451, 180463),
 (181870, 181882),
 (189897, 189909),
 (189967, 189979),
 (191597, 191609),
 (193287, 193299),
 (194384, 194396),
 (194716, 194728),
 (200012, 200024),
 (202615, 202627),
 (204152, 204164),
 (216202, 216214),
 (223004, 223016),
 (223782, 223794),
 (224308, 224320),
 (225342, 225354),
 (227868, 227880),
 (230726, 230738),
 (230766, 230778),
 (231028, 231040),
 (231126, 231138),
 (231958, 231970),
 (232006, 232018),
 (232113, 232125),
 (233202, 233214),
 (234592, 234604),
 (235120, 235132),
 (235580, 235592),
 (237533, 237545),
 (238111, 238123),
 (238581, 238593),
 (238737, 238749),
 (240296, 240308),
 (241817, 241829),
 (242242, 242254),
 (243664, 243676),
 (245754, 245766),
 (247515, 247527),
 (248702, 248714),
 (249294, 249306),
 (249417, 249429),
 (249650, 249662),
 (252042, 252054),
 (256738, 256750),
 (263439, 263451),
 (263522, 263534),
 (2